In [1]:
# Transformer 생성형 모델 & 파인 튜닝
# - Hugging Face 라이브러리 적용
# - AI HUB 금융 분야 다국어 병렬 말뭉치 데이터셋 적용
# - 입력된 한국어 문장을 생성형 모델을 한국어 문장 생성
# 1. 학습 목표
# - 구조 최적화 및 파이프라인 단순화
# - AI HUB 금융 분야 다국어 병렬 말뭉치 데이터셋 전처리
# - 금융 학술논문 데이터셋 변환 전처리
# - 토크나이징 및 토크나이징 전처리
# - 베이스 모델 로드
# - LoRA(Low-Rank Adaptation) 설정, 특정 레이어에 작은 저차원 행렬(랭크 r)을 삽입해서 학습
# - LoRA(Low-Rank Adaptation) 모델, 메모리 효율성/빠른 학습/도메인 적용, base 모델에 여러 LoRA 모듈을 붙였다 떼었다 할 수 있음
# - 학습 args 설정
# - Trainer 정의
# - Trainer 실행
# - LoRA 적용된 모델 저장, LoRA모델/토크나이저
# - LoRA 적용된 모델 불러오기, 베이스모델/LoRA모델/토크나이저

In [2]:
# 데이터셋 전처리 - AI HUB 금융 분야 다국어 병렬 말뭉치 데이터셋
import os
import glob
import json
import re

ko_lines = []
folders = [
    './llm_data/ai_hub_article/*.json'
]

# 모든 JSON 읽기
for folder in folders:
    for path in glob.glob(folder): # 특정 디렉토리에서 지정한 패턴과 일치하는 모든 파일 경로를 리스트로 반환
        with open(path, encoding='utf-8') as f:
            try:
                data = json.load(f) # 파일 전체 로드(dict 구조)
            except json.JSONDecodeError:
                continue
        
        # sents 리스트에서 문장 추출
        for sent in data.get('sents', []):
            sentences = sent.get('source_cleaned') # 최종번역문(한국어) 추출
            if sentences != 'N/A':
                ko_lines.append(sentences.strip())

# 특수문자/공백 전처리
def detokenize_sentence(sentence: str) -> str:
    sentence = sentence.strip() # 양쪽 공백 처리
    sentence = re.sub(r"\s+([?.!,])", r"\1", sentence)  # " ?" → "?"
    sentence = re.sub(r"\s+", " ", sentence)            # 여러 공백 → 하나
    return sentence

# 데이터셋 전처리
ko_lines = [ detokenize_sentence(s) for s in ko_lines ]

print(f'총 문장 개수: {len(ko_lines)}')
print(ko_lines[0])

총 문장 개수: 80883
2019년부터 전년말 기준 자산총액 2조 원 이상인 기업에 대해 의무화하였으며, 2020년부터는 전년말 기준 자산총액 5천 억 원 이상 기업까지 의무대상을 확대했다.


In [3]:
# 데이터 전처리 - 중복 문장 제거(set()), 너무 짧은 문장(3~4자) 제거, 너무 긴 문장 잘라내기(gpt-2는 max_length=1024 제한), 학습/검증/테스트 분리
from sklearn.model_selection import train_test_split

out_dir = './llm_data/ai_hub_article_llm'
# 폴더 없을시 생성
if not os.path.exists(out_dir):
    os.makedirs(out_dir, exist_ok=True)
    print(f'폴더 생성 완료: {out_dir}')
else:
    print(f'이미 존재하는 폴더: {out_dir}')

# 중복 제거
ko_lines = list(set(ko_lines))

# 최소 길이 필터링(10자 이상만 사용)
ko_lines = [ s for s in ko_lines if len(s) > 10 ]

# train/validation/test 분리
train, test = train_test_split(ko_lines, test_size=0.1, random_state=42)
train, val = train_test_split(train, test_size=0.1, random_state=42)

# 저장
with open(f'{out_dir}/train.txt', 'w', encoding='utf-8') as f: # train
    for line in train:
        f.write(line + '\n')
with open(f'{out_dir}/val.txt', 'w', encoding='utf-8') as f: # validation
    for line in val:
        f.write(line + '\n')
with open(f'{out_dir}/test.txt', 'w', encoding='utf-8') as f: # test
    for line in test:
        f.write(line + '\n')

print(f'총 문장 개수: {len(ko_lines)}')        
print('데이터셋 저장 완료')

이미 존재하는 폴더: ./llm_data/ai_hub_article_llm
총 문장 개수: 80868
데이터셋 저장 완료


In [86]:
# 데이터셋 생성
from datasets import load_dataset, DatasetDict

dataset = load_dataset(
    'text',
    data_files={
        'train': './llm_data/ai_hub_article_llm/train.txt',
        'validation': './llm_data/ai_hub_article_llm/val.txt',
        'test': './llm_data/ai_hub_article_llm/test.txt'
    }
)

# train/validation/test 각각에서 100개만 추출
small_train = dataset['train'].select(range(20000))
small_val = dataset['validation'].select(range(2000))
small_test = dataset['test'].select(range(2000))

small_dataset = DatasetDict({
    'train': small_train,
    'validation': small_val,
    'test': small_test
})

print(small_dataset['train'][0])

{'text': '본 연구는 이 모형을 통해 적합한 자산배분을 제시하려는 것이 아니라, 다양한 자본시장 환경을 감안하여 현재 재정추계에서 가정하고 있는 요구수익률이 합리적인지를 점검하는 데 일차적 목적을 두었기 때문이다.'}


In [87]:
# 토크나이저 및 데이터셋 생성
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('gpt2')
# 별도의 [PAD] 토큰을 추가, 모델의 임베딩을 확장해야 하므로, 학습 전에 반드시 model.resize_token_embeddings(len(tokenizer)) 호출 필요
tokenizer.add_special_tokens( {'pad_token': '[PAD]'} )

def tokenize_function(examples):
    tokens = tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=128
    )
    tokens['labels'] = tokens['input_ids'].copy()
    return tokens

# tokenized_dataset = dataset.map(tokenize_function, batched=True)
# print(tokenized_dataset['train'][0])

tokenized_dataset = small_dataset.map(tokenize_function, batched=True)
print(small_dataset['train'][0])

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'text': '본 연구는 이 모형을 통해 적합한 자산배분을 제시하려는 것이 아니라, 다양한 자본시장 환경을 감안하여 현재 재정추계에서 가정하고 있는 요구수익률이 합리적인지를 점검하는 데 일차적 목적을 두었기 때문이다.'}


In [89]:
# 모델 로드, 임베딩 확장, LoRA 설정
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

# 모델 로드 및 임베딩 확장
model = AutoModelForCausalLM.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))

# LoRA 설정
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=['c_attn'], # GPT-2 Attention 모듈

    lora_dropout=0.1,
    bias='none',
    task_type='CAUSAL_LM'
)
# LoRA 모델 생성
model = get_peft_model(model, lora_config)

In [90]:
# 학습 파라미터 정의
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# 학습 설정
training_args = TrainingArguments(
    output_dir="./llm_models/results_lora_llm_gpt2",    # 학습 결과(모델, 체크포인트) 저장 경로
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./llm_models/results_lora_logs_llm_gpt2",
    logging_steps=100,
    save_strategy="epoch",
    eval_strategy="epoch",
    fp16=True,  # GPU 메모리 절약
    learning_rate=5e-5,   # ← 학습률 추가
    load_best_model_at_end=True                     # 최적 모델 자동 로드

)

# Trainer 정의
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # 개선 없으면 조기 종료
)

In [91]:
# 학습 시작
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.728300,2.653009
2,2.645700,2.600876
3,2.686600,2.587525


d:\AI\Pytorch\deep-learning\.venv\Lib\site-packages\peft\utils\save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
d:\AI\Pytorch\deep-learning\.venv\Lib\site-packages\peft\utils\save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
d:\AI\Pytorch\deep-learning\.venv\Lib\site-packages\peft\utils\save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=3750, training_loss=2.7541090494791667, metrics={'train_runtime': 4456.6574, 'train_samples_per_second': 13.463, 'train_steps_per_second': 0.841, 'total_flos': 3932970024960000.0, 'train_loss': 2.7541090494791667, 'epoch': 3.0})

In [92]:
# LoRA 모델 저장
model.save_pretrained("./llm_models/llm_gpt2_lora", safe_serialization=False)

# 토크나이저 저장
tokenizer.save_pretrained("./llm_models/llm_gpt2_lora")


d:\AI\Pytorch\deep-learning\.venv\Lib\site-packages\peft\utils\save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


('./llm_models/llm_gpt2_lora\\tokenizer_config.json',
 './llm_models/llm_gpt2_lora\\special_tokens_map.json',
 './llm_models/llm_gpt2_lora\\vocab.json',
 './llm_models/llm_gpt2_lora\\merges.txt',
 './llm_models/llm_gpt2_lora\\added_tokens.json',
 './llm_models/llm_gpt2_lora\\tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel

# 1) 토크나이저: 학습 때 저장한 것을 불러옴 (vocab=50258)
tokenizer = AutoTokenizer.from_pretrained("./llm_models/llm_gpt2_lora")

# 2) base GPT-2 불러오고, 임베딩을 50258로 확장
base_model = AutoModelForCausalLM.from_pretrained("gpt2")
base_model.resize_token_embeddings(len(tokenizer))  # 50258로 확장

# 3) LoRA 어댑터를 base_model 위에 로드
model = PeftModel.from_pretrained(base_model, "./llm_models/llm_gpt2_lora")

# 4) 파이프라인으로 생성 테스트
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "2025년 금융 규제 변화에 따라"
outputs = generator(
    prompt,
    max_new_tokens=120,
    do_sample=True,
    top_k=50,
    top_p=0.9,
    temperature=0.7,
    repetition_penalty=1.2,   # 반복 억제
    no_repeat_ngram_size=3    # 3-gram 반복 방지
)

print(outputs[0]["generated_text"])

Device set to use cuda:0
